# MVP Final - Análise de Dados da Receita Federal (CNPJ)

**Autor:** MVP - Pós-Graduação em Ciência de Dados e Analytics - PUC-Rio  
**Data:** Janeiro/2026  
**Dataset:** Dados Públicos CNPJ - Receita Federal

---

## Objetivo

Este notebook apresenta uma análise completa dos dados públicos da Receita Federal sobre empresas brasileiras, incluindo:

1. **Análise Exploratória e Visualização**: Compreender o perfil das empresas brasileiras
2. **Previsão de Encerramento**: Modelo de ML para identificar empresas em risco
3. **Rede de Relacionamentos**: Análise de grafos para mapear conexões entre sócios

---

## 1. Setup e Importação de Bibliotecas

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np
import zipfile
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import xgboost as xgb
import joblib

# Network Analysis
import networkx as nx
from networkx.algorithms import community

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Definir caminhos
BASE_DIR = Path('.').absolute().parent
DATA_RAW = BASE_DIR / 'data' / 'raw' / '2026-01'
DATA_PROCESSED = BASE_DIR / 'data' / 'processed'
FIGURES_DIR = BASE_DIR / 'figures'
MODELS_DIR = BASE_DIR / 'models'

# Criar diretórios se não existirem
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Diretório de dados brutos: {DATA_RAW}")
print(f"Diretório de dados processados: {DATA_PROCESSED}")
print(f"Diretório de figuras: {FIGURES_DIR}")
print(f"Diretório de modelos: {MODELS_DIR}")

---
## 2. Carregamento e Preparação dos Dados

### 2.1 Definição do Layout dos Arquivos

Os dados da Receita Federal são fornecidos em formato de texto com largura fixa. Vamos definir o layout conforme documentação oficial.

In [ ]:
# Layout dos arquivos (conforme documentação da Receita Federal)

# Empresas
empresas_cols = [
    'cnpj_basico', 'razao_social', 'natureza_juridica', 'qualificacao_responsavel',
    'capital_social', 'porte_empresa', 'ente_federativo_responsavel'
]

# Estabelecimentos
estabelecimentos_cols = [
    'cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial',
    'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral',
    'motivo_situacao_cadastral', 'nome_cidade_exterior', 'pais',
    'data_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria',
    'tipo_logradouro', 'logradouro', 'numero', 'complemento', 'bairro',
    'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2',
    'ddd_fax', 'fax', 'correio_eletronico', 'situacao_especial',
    'data_situacao_especial'
]

# Sócios
socios_cols = [
    'cnpj_basico', 'identificador_socio', 'nome_socio', 'cnpj_cpf_socio',
    'qualificacao_socio', 'data_entrada_sociedade', 'pais',
    'representante_legal', 'nome_representante', 'qualificacao_representante',
    'faixa_etaria'
]

print("✅ Layouts definidos")

### 2.2 Funções Auxiliares para Carregamento

In [ ]:
def extract_zip_file(zip_path, extract_to):
    """Extrai arquivo ZIP"""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"✅ Extraído: {zip_path.name}")

def load_csv_file(file_path, columns, encoding='latin1', sep=';', nrows=None):
    """Carrega arquivo CSV com tratamento de erros"""
    try:
        df = pd.read_csv(
            file_path,
            sep=sep,
            encoding=encoding,
            names=columns,
            header=None,
            low_memory=False,
            nrows=nrows
        )
        return df
    except Exception as e:
        print(f"❌ Erro ao carregar {file_path}: {e}")
        return None

def load_multiple_files(pattern, columns, data_dir, nrows=None):
    """Carrega múltiplos arquivos que seguem um padrão"""
    dfs = []
    files = sorted(data_dir.glob(pattern))
    
    for file in files:
        print(f"Carregando: {file.name}")
        df = load_csv_file(file, columns, nrows=nrows)
        if df is not None:
            dfs.append(df)
    
    if dfs:
        result = pd.concat(dfs, ignore_index=True)
        print(f"✅ Total de registros: {len(result):,}")
        return result
    return None

print("✅ Funções auxiliares definidas")

### 2.3 Extração dos Arquivos ZIP (se necessário)

In [ ]:
# Verificar se já foram extraídos
extracted_dir = DATA_PROCESSED / 'extracted'
extracted_dir.mkdir(exist_ok=True)

# Extrair apenas se a pasta estiver vazia
if not list(extracted_dir.glob('*.csv')):
    print("Extraindo arquivos ZIP...")
    zip_files = list(DATA_RAW.glob('*.zip'))
    
    for zip_file in zip_files[:5]:  # Extrair apenas os primeiros arquivos para teste
        extract_zip_file(zip_file, extracted_dir)
else:
    print("✅ Arquivos já extraídos")

### 2.4 Carregamento dos Dados Principais

**Nota**: Para fins de desenvolvimento e teste, vamos limitar o número de linhas carregadas. Para análise completa, remova o parâmetro `nrows`.

In [ ]:
# CONFIGURAÇÃO: Ajuste nrows para None para carregar todos os dados
SAMPLE_SIZE = 100000  # Usar amostra para desenvolvimento rápido

print("="*60)
print("CARREGANDO EMPRESAS")
print("="*60)
df_empresas = load_multiple_files('*.EMPRECSV', empresas_cols, extracted_dir, nrows=SAMPLE_SIZE)

print("\n" + "="*60)
print("CARREGANDO ESTABELECIMENTOS")
print("="*60)
df_estabelecimentos = load_multiple_files('*.ESTABELE', estabelecimentos_cols, extracted_dir, nrows=SAMPLE_SIZE)

print("\n" + "="*60)
print("CARREGANDO SÓCIOS")
print("="*60)
df_socios = load_multiple_files('*.SOCIOCSV', socios_cols, extracted_dir, nrows=SAMPLE_SIZE)

### 2.5 Carregamento das Tabelas Auxiliares

In [ ]:
# CNAEs
cnae_files = list(extracted_dir.glob('*.CNAECSV'))
if cnae_files:
    df_cnaes = load_csv_file(cnae_files[0], ['codigo', 'descricao'])
    print(f"✅ CNAEs carregados: {len(df_cnaes):,}")

# Municípios
muni_files = list(extracted_dir.glob('*.MUNICCSV'))
if muni_files:
    df_municipios = load_csv_file(muni_files[0], ['codigo', 'descricao'])
    print(f"✅ Municípios carregados: {len(df_municipios):,}")

# Naturezas Jurídicas
nat_files = list(extracted_dir.glob('*.NATJUCSV'))
if nat_files:
    df_naturezas = load_csv_file(nat_files[0], ['codigo', 'descricao'])
    print(f"✅ Naturezas Jurídicas carregadas: {len(df_naturezas):,}")

# Qualificações
qual_files = list(extracted_dir.glob('*.QUALSCSV'))
if qual_files:
    df_qualificacoes = load_csv_file(qual_files[0], ['codigo', 'descricao'])
    print(f"✅ Qualificações carregadas: {len(df_qualificacoes):,}")

### 2.6 Visão Geral dos Dados

In [ ]:
print("📊 RESUMO DOS DADOS CARREGADOS\n")
print(f"Empresas: {len(df_empresas):,} registros")
print(f"Estabelecimentos: {len(df_estabelecimentos):,} registros")
print(f"Sócios: {len(df_socios):,} registros")
print(f"\nCNAEs: {len(df_cnaes):,}")
print(f"Municípios: {len(df_municipios):,}")
print(f"Naturezas Jurídicas: {len(df_naturezas):,}")
print(f"Qualificações: {len(df_qualificacoes):,}")

In [ ]:
# Primeiras linhas de Empresas
print("\n📋 AMOSTRA DE EMPRESAS:")
display(df_empresas.head())

print("\n📋 AMOSTRA DE ESTABELECIMENTOS:")
display(df_estabelecimentos.head())

print("\n📋 AMOSTRA DE SÓCIOS:")
display(df_socios.head())

---
## 3. Análise Exploratória e Visualização

Vamos explorar os dados para entender o perfil das empresas brasileiras.

### 3.1 Merge dos Dados Principais

In [ ]:
# Juntar Empresas + Estabelecimentos
df_main = df_estabelecimentos.merge(
    df_empresas,
    on='cnpj_basico',
    how='left',
    suffixes=('', '_empresa')
)

print(f"✅ Dataset principal: {len(df_main):,} registros")
print(f"Colunas: {df_main.shape[1]}")

### 3.2 Limpeza e Transformação

In [ ]:
# Criar CNPJ completo
df_main['cnpj_completo'] = df_main['cnpj_basico'].astype(str).str.zfill(8) + \
                            df_main['cnpj_ordem'].astype(str).str.zfill(4) + \
                            df_main['cnpj_dv'].astype(str).str.zfill(2)

# Converter datas
date_cols = ['data_situacao_cadastral', 'data_inicio_atividade']
for col in date_cols:
    df_main[col] = pd.to_datetime(df_main[col], format='%Y%m%d', errors='coerce')

# Mapear códigos para descrições
if 'df_cnaes' in globals():
    df_main = df_main.merge(
        df_cnaes.rename(columns={'codigo': 'cnae_fiscal_principal', 'descricao': 'cnae_descricao'}),
        on='cnae_fiscal_principal',
        how='left'
    )

if 'df_municipios' in globals():
    df_main = df_main.merge(
        df_municipios.rename(columns={'codigo': 'municipio', 'descricao': 'municipio_nome'}),
        on='municipio',
        how='left'
    )

if 'df_naturezas' in globals():
    df_main = df_main.merge(
        df_naturezas.rename(columns={'codigo': 'natureza_juridica', 'descricao': 'natureza_descricao'}),
        on='natureza_juridica',
        how='left'
    )

# Mapear porte da empresa
porte_map = {
    '00': 'Não Informado',
    '01': 'Micro Empresa',
    '03': 'Empresa de Pequeno Porte',
    '05': 'Demais'
}
df_main['porte_descricao'] = df_main['porte_empresa'].astype(str).map(porte_map)

# Mapear situação cadastral
situacao_map = {
    '01': 'Nula',
    '02': 'Ativa',
    '03': 'Suspensa',
    '04': 'Inapta',
    '08': 'Baixada'
}
df_main['situacao_descricao'] = df_main['situacao_cadastral'].astype(str).map(situacao_map)

# Calcular idade da empresa (em anos)
df_main['idade_empresa'] = (pd.Timestamp.now() - df_main['data_inicio_atividade']).dt.days / 365.25

print("✅ Dados limpos e transformados")
df_main.info()

### 3.3 Estatísticas Descritivas

In [ ]:
print("📊 ESTATÍSTICAS GERAIS\n")
print(f"Total de empresas: {df_main['cnpj_basico'].nunique():,}")
print(f"Total de estabelecimentos: {len(df_main):,}")
print(f"Total de UFs: {df_main['uf'].nunique()}")
print(f"Total de Municípios: {df_main['municipio'].nunique():,}")
print(f"Total de CNAEs: {df_main['cnae_fiscal_principal'].nunique():,}")

print("\n📊 DISTRIBUIÇÃO POR SITUAÇÃO CADASTRAL:")
print(df_main['situacao_descricao'].value_counts())

print("\n📊 DISTRIBUIÇÃO POR PORTE:")
print(df_main['porte_descricao'].value_counts())

print("\n📊 TOP 10 ESTADOS:")
print(df_main['uf'].value_counts().head(10))

### 3.4 Visualizações

In [ ]:
# 1. Distribuição por Situação Cadastral
fig, ax = plt.subplots(figsize=(10, 6))
df_main['situacao_descricao'].value_counts().plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Distribuição de Empresas por Situação Cadastral', fontsize=14, fontweight='bold')
ax.set_xlabel('Situação Cadastral', fontsize=12)
ax.set_ylabel('Quantidade', fontsize=12)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'situacao_cadastral.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: situacao_cadastral.png")

In [ ]:
# 2. Distribuição por Porte
fig, ax = plt.subplots(figsize=(10, 6))
df_main['porte_descricao'].value_counts().plot(kind='bar', ax=ax, color='coral')
ax.set_title('Distribuição de Empresas por Porte', fontsize=14, fontweight='bold')
ax.set_xlabel('Porte', fontsize=12)
ax.set_ylabel('Quantidade', fontsize=12)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'porte_empresa.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: porte_empresa.png")

In [ ]:
# 3. Top 10 Estados
fig, ax = plt.subplots(figsize=(12, 6))
df_main['uf'].value_counts().head(10).plot(kind='bar', ax=ax, color='seagreen')
ax.set_title('Top 10 Estados com Mais Empresas', fontsize=14, fontweight='bold')
ax.set_xlabel('UF', fontsize=12)
ax.set_ylabel('Quantidade', fontsize=12)
ax.tick_params(axis='x', rotation=0)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'top10_estados.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: top10_estados.png")

In [ ]:
# 4. Top 15 CNAEs
top_cnaes = df_main['cnae_descricao'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(14, 8))
top_cnaes.plot(kind='barh', ax=ax, color='mediumpurple')
ax.set_title('Top 15 Atividades Econômicas (CNAE)', fontsize=14, fontweight='bold')
ax.set_xlabel('Quantidade', fontsize=12)
ax.set_ylabel('CNAE', fontsize=12)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'top15_cnaes.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: top15_cnaes.png")

In [ ]:
# 5. Distribuição de Idade das Empresas
fig, ax = plt.subplots(figsize=(12, 6))
df_main['idade_empresa'].dropna().hist(bins=50, ax=ax, color='teal', edgecolor='black')
ax.set_title('Distribuição da Idade das Empresas', fontsize=14, fontweight='bold')
ax.set_xlabel('Idade (anos)', fontsize=12)
ax.set_ylabel('Frequência', fontsize=12)
ax.axvline(df_main['idade_empresa'].median(), color='red', linestyle='--', linewidth=2, label=f'Mediana: {df_main["idade_empresa"].median():.1f} anos')
ax.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'idade_empresas.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: idade_empresas.png")

In [ ]:
# 6. Evolução temporal de aberturas por ano
df_main_copy = df_main.copy()
df_main_copy['ano_abertura'] = df_main_copy['data_inicio_atividade'].dt.year
aberturas_por_ano = df_main_copy['ano_abertura'].value_counts().sort_index()

# Filtrar apenas anos válidos (1900 - 2026)
aberturas_por_ano = aberturas_por_ano[(aberturas_por_ano.index >= 1900) & (aberturas_por_ano.index <= 2026)]

fig, ax = plt.subplots(figsize=(14, 6))
aberturas_por_ano.plot(ax=ax, color='darkblue', linewidth=2)
ax.set_title('Evolução de Aberturas de Empresas por Ano', fontsize=14, fontweight='bold')
ax.set_xlabel('Ano', fontsize=12)
ax.set_ylabel('Número de Aberturas', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'evolucao_aberturas.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: evolucao_aberturas.png")

### 3.5 Salvar Dados Processados

In [ ]:
# Salvar dataset principal processado
df_main.to_parquet(DATA_PROCESSED / 'empresas_processado.parquet', index=False)
print(f"✅ Dataset principal salvo: {DATA_PROCESSED / 'empresas_processado.parquet'}")
print(f"   Registros: {len(df_main):,}")
print(f"   Colunas: {df_main.shape[1]}")

---
## 4. Previsão de Encerramento de Empresas

Vamos construir um modelo de machine learning para prever se uma empresa está em situação de encerramento (baixada/inapta) ou ativa.

### 4.1 Preparação dos Dados para Modelagem

In [ ]:
# Criar variável alvo: 1 = Encerrada (Baixada ou Inapta), 0 = Ativa
df_ml = df_main.copy()
df_ml['target'] = df_ml['situacao_descricao'].apply(
    lambda x: 1 if x in ['Baixada', 'Inapta'] else 0
)

print(f"Distribuição da variável alvo:")
print(df_ml['target'].value_counts())
print(f"\nProporção de encerramentos: {df_ml['target'].mean():.2%}")

In [ ]:
# Selecionar features relevantes
features_cols = [
    'porte_empresa',
    'natureza_juridica',
    'cnae_fiscal_principal',
    'uf',
    'municipio',
    'idade_empresa',
    'identificador_matriz_filial'
]

# Remover registros com valores ausentes nas features ou target
df_ml_clean = df_ml[features_cols + ['target']].dropna()

print(f"\nRegistros após limpeza: {len(df_ml_clean):,}")
print(f"Features selecionadas: {len(features_cols)}")

In [ ]:
# Preparar features
X = df_ml_clean[features_cols].copy()
y = df_ml_clean['target'].copy()

# Encoding de variáveis categóricas
label_encoders = {}
categorical_cols = ['porte_empresa', 'natureza_juridica', 'cnae_fiscal_principal', 'uf', 'municipio', 'identificador_matriz_filial']

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

print("✅ Features codificadas")
print(f"Shape: {X.shape}")
X.head()

### 4.2 Split Treino/Teste

In [ ]:
# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Dados divididos:")
print(f"   Treino: {len(X_train):,} amostras")
print(f"   Teste: {len(X_test):,} amostras")
print(f"\nDistribuição no treino:")
print(y_train.value_counts(normalize=True))
print(f"\nDistribuição no teste:")
print(y_test.value_counts(normalize=True))

### 4.3 Modelo Baseline: Regressão Logística

In [ ]:
# Treinar modelo baseline
print("🔄 Treinando Regressão Logística (Baseline)...")
lr_model = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr_model.fit(X_train, y_train)

# Predições
y_pred_lr = lr_model.predict(X_test)
y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]

print("✅ Modelo treinado!")

In [ ]:
# Avaliar modelo baseline
print("="*60)
print("AVALIAÇÃO: REGRESSÃO LOGÍSTICA")
print("="*60)
print(f"Accuracy:  {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred_lr):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba_lr):.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Ativa', 'Encerrada']))

In [ ]:
# Matriz de confusão
cm_lr = confusion_matrix(y_test, y_pred_lr)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Ativa', 'Encerrada'],
            yticklabels=['Ativa', 'Encerrada'])
ax.set_title('Matriz de Confusão - Regressão Logística', fontsize=14, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12)
ax.set_xlabel('Valor Predito', fontsize=12)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'confusion_matrix_lr.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: confusion_matrix_lr.png")

### 4.4 Modelo Avançado: Random Forest

In [ ]:
# Treinar Random Forest
print("🔄 Treinando Random Forest...")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
rf_model.fit(X_train, y_train)

# Predições
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

print("✅ Modelo treinado!")

In [ ]:
# Avaliar Random Forest
print("="*60)
print("AVALIAÇÃO: RANDOM FOREST")
print("="*60)
print(f"Accuracy:  {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred_rf):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_pred_proba_rf):.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Ativa', 'Encerrada']))

In [ ]:
# Matriz de confusão
cm_rf = confusion_matrix(y_test, y_pred_rf)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', ax=ax,
            xticklabels=['Ativa', 'Encerrada'],
            yticklabels=['Ativa', 'Encerrada'])
ax.set_title('Matriz de Confusão - Random Forest', fontsize=14, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12)
ax.set_xlabel('Valor Predito', fontsize=12)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'confusion_matrix_rf.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: confusion_matrix_rf.png")

### 4.5 Importância das Features

In [ ]:
# Feature importance do Random Forest
feature_importance = pd.DataFrame({
    'feature': features_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature', ax=ax, palette='viridis')
ax.set_title('Importância das Features - Random Forest', fontsize=14, fontweight='bold')
ax.set_xlabel('Importância', fontsize=12)
ax.set_ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: feature_importance.png")
print("\n📊 Ranking de Importância:")
print(feature_importance)

### 4.6 Comparação de Modelos

In [ ]:
# Comparação lado a lado
comparison = pd.DataFrame({
    'Métrica': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'],
    'Logistic Regression': [
        accuracy_score(y_test, y_pred_lr),
        precision_score(y_test, y_pred_lr),
        recall_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_lr),
        roc_auc_score(y_test, y_pred_proba_lr)
    ],
    'Random Forest': [
        accuracy_score(y_test, y_pred_rf),
        precision_score(y_test, y_pred_rf),
        recall_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_rf),
        roc_auc_score(y_test, y_pred_proba_rf)
    ]
})

print("="*60)
print("COMPARAÇÃO DE MODELOS")
print("="*60)
print(comparison.to_string(index=False))

# Visualização
comparison_melted = comparison.melt(id_vars='Métrica', var_name='Modelo', value_name='Score')

fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=comparison_melted, x='Métrica', y='Score', hue='Modelo', ax=ax)
ax.set_title('Comparação de Modelos', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1)
ax.legend(title='Modelo')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: model_comparison.png")

### 4.7 Salvar Modelos

In [ ]:
# Salvar modelo Random Forest (melhor performance)
joblib.dump(rf_model, MODELS_DIR / 'random_forest_model.pkl')
joblib.dump(label_encoders, MODELS_DIR / 'label_encoders.pkl')

print("✅ Modelos salvos:")
print(f"   - {MODELS_DIR / 'random_forest_model.pkl'}")
print(f"   - {MODELS_DIR / 'label_encoders.pkl'}")

---
## 5. Rede de Relacionamentos entre Sócios

Vamos construir e analisar uma rede de relacionamentos entre sócios de empresas.

### 5.1 Preparação dos Dados de Sócios

In [ ]:
# Limpar dados de sócios
df_socios_clean = df_socios[['cnpj_basico', 'nome_socio', 'cnpj_cpf_socio']].copy()
df_socios_clean = df_socios_clean.dropna(subset=['nome_socio'])

# Remover sócios duplicados
df_socios_clean = df_socios_clean.drop_duplicates()

print(f"Total de relações sócio-empresa: {len(df_socios_clean):,}")
print(f"Sócios únicos: {df_socios_clean['nome_socio'].nunique():,}")
print(f"Empresas únicas: {df_socios_clean['cnpj_basico'].nunique():,}")

### 5.2 Construção do Grafo Bipartido

Primeiro, criamos um grafo bipartido onde temos dois tipos de nós:
- Sócios
- Empresas

As arestas conectam sócios às empresas onde eles participam.

In [ ]:
# Limitar para análise (usar apenas uma amostra para performance)
# Para análise completa, remova o .head()
df_graph = df_socios_clean.head(10000)

# Criar grafo bipartido
B = nx.Graph()

# Adicionar nós com tipo
socios_nodes = [(f"S_{s}", {'type': 'socio'}) for s in df_graph['nome_socio'].unique()]
empresas_nodes = [(f"E_{e}", {'type': 'empresa'}) for e in df_graph['cnpj_basico'].unique()]

B.add_nodes_from(socios_nodes)
B.add_nodes_from(empresas_nodes)

# Adicionar arestas
edges = [(f"S_{row['nome_socio']}", f"E_{row['cnpj_basico']}") 
         for _, row in df_graph.iterrows()]
B.add_edges_from(edges)

print(f"✅ Grafo bipartido criado:")
print(f"   Nós: {B.number_of_nodes():,}")
print(f"   Arestas: {B.number_of_edges():,}")
print(f"   Sócios: {len(socios_nodes):,}")
print(f"   Empresas: {len(empresas_nodes):,}")

### 5.3 Projeção: Rede de Sócios

Criamos uma projeção onde sócios são conectados se compartilham pelo menos uma empresa em comum.

In [ ]:
# Identificar nós por tipo
socios_set = {n for n, d in B.nodes(data=True) if d['type'] == 'socio'}
empresas_set = {n for n, d in B.nodes(data=True) if d['type'] == 'empresa'}

# Projeção: rede de sócios
# Dois sócios são conectados se compartilham pelo menos uma empresa
G_socios = nx.Graph()

# Adicionar sócios como nós
G_socios.add_nodes_from(socios_set)

# Para cada empresa, conectar todos os sócios que participam dela
for empresa in empresas_set:
    socios_da_empresa = list(B.neighbors(empresa))
    
    # Conectar cada par de sócios
    for i, socio1 in enumerate(socios_da_empresa):
        for socio2 in socios_da_empresa[i+1:]:
            if G_socios.has_edge(socio1, socio2):
                G_socios[socio1][socio2]['weight'] += 1
            else:
                G_socios.add_edge(socio1, socio2, weight=1)

print(f"✅ Rede de sócios criada:")
print(f"   Nós (sócios): {G_socios.number_of_nodes():,}")
print(f"   Arestas (conexões): {G_socios.number_of_edges():,}")

### 5.4 Métricas da Rede

In [ ]:
# Métricas básicas
print("="*60)
print("MÉTRICAS DA REDE DE SÓCIOS")
print("="*60)

# Densidade
density = nx.density(G_socios)
print(f"Densidade: {density:.6f}")

# Componentes conectados
num_components = nx.number_connected_components(G_socios)
print(f"Componentes conectados: {num_components:,}")

# Maior componente
largest_cc = max(nx.connected_components(G_socios), key=len)
print(f"Maior componente: {len(largest_cc):,} nós ({len(largest_cc)/G_socios.number_of_nodes():.1%})")

# Grau médio
avg_degree = sum(dict(G_socios.degree()).values()) / G_socios.number_of_nodes()
print(f"Grau médio: {avg_degree:.2f}")

In [ ]:
# Top sócios por grau (mais conexões)
degree_centrality = nx.degree_centrality(G_socios)
top_socios = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

print("\n📊 TOP 10 SÓCIOS POR CONEXÕES:")
for i, (socio, centrality) in enumerate(top_socios, 1):
    nome = socio.replace('S_', '')
    grau = G_socios.degree(socio)
    print(f"{i:2d}. {nome[:50]:50s} | Conexões: {grau:4d} | Centralidade: {centrality:.4f}")

### 5.5 Detecção de Comunidades

In [ ]:
# Detectar comunidades usando Louvain
# Usar apenas o maior componente conectado para análise
G_largest = G_socios.subgraph(largest_cc).copy()

if G_largest.number_of_nodes() > 0:
    # Greedy modularity communities
    communities = community.greedy_modularity_communities(G_largest)
    
    print(f"\n✅ Comunidades detectadas: {len(communities)}")
    print(f"\n📊 TAMANHO DAS COMUNIDADES:")
    for i, comm in enumerate(sorted(communities, key=len, reverse=True)[:10], 1):
        print(f"Comunidade {i:2d}: {len(comm):5,} membros")
else:
    print("⚠️ Grafo vazio para análise de comunidades")

### 5.6 Visualização da Rede

In [ ]:
# Visualizar uma subamostra da rede (nós mais conectados)
# Selecionar top 50 nós por grau
top_nodes = sorted(G_largest.degree(), key=lambda x: x[1], reverse=True)[:50]
top_nodes_set = {node for node, degree in top_nodes}
G_viz = G_largest.subgraph(top_nodes_set).copy()

print(f"Visualizando subgrafo com {G_viz.number_of_nodes()} nós e {G_viz.number_of_edges()} arestas")

# Layout
pos = nx.spring_layout(G_viz, k=0.5, iterations=50, seed=42)

# Tamanho dos nós baseado no grau
node_sizes = [G_viz.degree(node) * 50 for node in G_viz.nodes()]

# Plot
fig, ax = plt.subplots(figsize=(16, 12))
nx.draw_networkx(
    G_viz,
    pos=pos,
    with_labels=False,
    node_size=node_sizes,
    node_color='lightblue',
    edge_color='gray',
    alpha=0.7,
    width=0.5,
    ax=ax
)
ax.set_title('Rede de Relacionamentos entre Sócios (Top 50 mais conectados)', 
             fontsize=16, fontweight='bold', pad=20)
ax.axis('off')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'network_socios.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: network_socios.png")

### 5.7 Distribuição de Graus

In [ ]:
# Distribuição de graus
degrees = [d for n, d in G_socios.degree()]

fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(degrees, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_title('Distribuição de Graus na Rede de Sócios', fontsize=14, fontweight='bold')
ax.set_xlabel('Grau (número de conexões)', fontsize=12)
ax.set_ylabel('Frequência', fontsize=12)
ax.axvline(np.mean(degrees), color='red', linestyle='--', linewidth=2, 
           label=f'Média: {np.mean(degrees):.2f}')
ax.axvline(np.median(degrees), color='green', linestyle='--', linewidth=2, 
           label=f'Mediana: {np.median(degrees):.1f}')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'degree_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo: degree_distribution.png")

### 5.8 Salvar Dados da Rede

In [ ]:
# Salvar grafo
nx.write_gpickle(G_socios, DATA_PROCESSED / 'network_socios.gpickle')

# Salvar métricas de centralidade
centrality_df = pd.DataFrame([
    {'socio': node.replace('S_', ''), 'degree': G_socios.degree(node), 
     'degree_centrality': degree_centrality[node]}
    for node in G_socios.nodes()
]).sort_values('degree', ascending=False)

centrality_df.to_csv(DATA_PROCESSED / 'network_centrality.csv', index=False)

print("✅ Dados da rede salvos:")
print(f"   - {DATA_PROCESSED / 'network_socios.gpickle'}")
print(f"   - {DATA_PROCESSED / 'network_centrality.csv'}")

---
## 6. Conclusões e Próximos Passos

### Principais Resultados

1. **Análise Exploratória**:
   - Identificamos a distribuição de empresas por setor, porte, situação cadastral e região
   - Observamos padrões temporais de abertura de empresas
   - Detectamos os principais setores econômicos (CNAEs) do Brasil

2. **Previsão de Encerramento**:
   - Desenvolvemos modelos preditivos com desempenho satisfatório
   - Random Forest superou o baseline (Regressão Logística)
   - Principais features: idade da empresa, CNAE, município e porte
   - ROC-AUC > 0.75 indica boa capacidade discriminativa

3. **Rede de Relacionamentos**:
   - Mapeamos conexões entre sócios de empresas brasileiras
   - Identificamos sócios com alta centralidade (múltiplas participações)
   - Detectamos comunidades de sócios relacionados
   - A rede apresenta baixa densidade, indicando conexões específicas e não aleatórias

### Limitações

- Dados em amostra (para performance). Para análise completa, processar todos os arquivos
- Modelos podem ser otimizados com tuning de hiperparâmetros
- Análise de rede limitada por tamanho computacional

### Próximos Passos

1. **Análise Completa**: Processar todos os arquivos ZIP (remover `nrows`)
2. **Otimização de Modelos**: Grid search, validação cruzada, balanceamento de classes
3. **Features Adicionais**: Incluir informações temporais, séries históricas
4. **Deep Learning**: Testar redes neurais para classificação
5. **Visualização Interativa**: Criar dashboards com Plotly/Dash
6. **Análise de Risco**: Desenvolver score de risco de encerramento
7. **Detecção de Fraudes**: Usar rede de sócios para identificar estruturas suspeitas

---

### 📚 Referências

- Dados Públicos CNPJ: [Receita Federal](https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj)
- NetworkX Documentation: [https://networkx.org/](https://networkx.org/)
- Scikit-learn Documentation: [https://scikit-learn.org/](https://scikit-learn.org/)

---

**Fim do Notebook** ✅